In [27]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col, col2im
#im2col(input data, filter_h, filter_w, stride, pad)
import numpy as np

In [7]:
x1 = np.random.rand(1,3,7,7) #데이터 수, 채널 수, 높이, 너비
print(x1)
col1 = im2col(x1, 5, 5, stride = 1, pad = 0)
print(col1.shape) #2차원

[[[[0.67569525 0.98778684 0.48278902 0.82936608 0.60702582 0.60151741
    0.83343899]
   [0.05058901 0.73408291 0.80630997 0.74976806 0.59267051 0.18131483
    0.96097326]
   [0.64118353 0.67682226 0.18246338 0.87305256 0.81578391 0.36802587
    0.18437514]
   [0.03042944 0.39159957 0.66876525 0.04027652 0.81540015 0.54348642
    0.35701328]
   [0.91888099 0.85574781 0.03320992 0.76028827 0.82481538 0.88936193
    0.96804112]
   [0.00145347 0.56365007 0.58643794 0.98002473 0.47034235 0.45681738
    0.08022506]
   [0.24865284 0.25724802 0.05020502 0.84156959 0.66879986 0.95354725
    0.83344954]]

  [[0.31067921 0.23041444 0.4118225  0.82120445 0.66410694 0.94560599
    0.98698606]
   [0.45213241 0.39303234 0.60120528 0.79196753 0.52954599 0.02958525
    0.23148093]
   [0.33642237 0.5823537  0.71503223 0.78411934 0.23809358 0.03872247
    0.95483029]
   [0.83313607 0.65355307 0.37680216 0.58821304 0.79061994 0.36200339
    0.72340819]
   [0.90644832 0.65053694 0.31786253 0.34727035 0.54

In [9]:
x2 = np.random.rand(10, 3, 7, 7)
#print(x2)
col2 = im2col(x2, 5, 5, stride = 1, pad=0)
print(col2.shape)

(90, 75)


In [30]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        #forward
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        #backward
        self.x = None
        self.col = None
        self.col_W = None
        
        #W, b의 기울기
        self.dW = None
        self.db = None
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape #필터 개수, 채널, 필터높이, 필터너비
        N, C, H, W = x.shape
        out_h = int(1+ (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W+ 2*self.pad - FW) / self.stride) #output 높이 너비
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # 필터 전개 : -1을 하면 원소 수 똑같도록 자동 설정
        out = np.dot(col, col_W) + self.b # x*W + b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2) #N, C, out_h, out_w 순서로 변경
        
        #backward 위해서
        self.x = x
        self.col = col
        self.col_W = col_W
        
        return out
    
    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0, 2, 3, 1).reshape(-1, FN) #N, H, W, C 순서 -> 개수, FN으로 reshape(거꾸로)
        
        self.db = np.sum(dout, axis=0)
        self.dw = np.dot(self.col.T, dout) #col의 전치행렬과 dout 곱
        self.dw = self.dw.transpose(1, 0).reshape(FN, C, FH, FW)
        
        dcol = np.dot(dout, self.col_W.T) #dout와 col_W의 전치행렬과의 곱
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)
        
        return dx

In [31]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride = 1, pad = 0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1+(H - self.pool_h) / self.stride)
        out_w = int(1+(W - self.pool_w) / self.stride)
        
        #im2col로 입력 데이터 전개
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w) #-1은 채널의 수가 될 것이다
        
        
        arg_max = np.argmax(col, axis=1) #argmax: max값에 해당하는 인덱스 번호
        
        #행별로 최댓값 구하기
        out = np.max(col, axis=1) # axis = 0: 열방향, axis = 1: 행방행 -> x의 1번째 차원의 축마다 최댓값
        
        #reshape
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2) #개수, 채널수, 높이, 너비 순서로
        
        #역전파용
        self.x = x
        self.arg_max = arg_max
        
        return out
    
    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size, ))
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx

In [32]:
from common.layers import Relu, Affine, SoftmaxWithLoss
from common.gradient import numerical_gradient
import pickle #텍스트 이외의 자료형(딕셔너리, 튜플, 리스트 등) 저장하기 위해 사용 - 텍스트는 파일 입출력쓰면 됨
from collections import OrderedDict

In [40]:
class SimpleConvNet:
    #conv - relu - pool - affine - relu - affine - softmax
    
    
    #초기화
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        #input_dim: 입력 데이터의 차원(C, H, W)
        #conv_param: 딕셔너리로 필터수, 크기 등 구현
        #weight_init_std: 가중치의 표준편차 - 0.01, 'he', or 'relu', 'xavier'  or 'sigmoid' 가능
        
        #편리함 위해 딕셔너리값 따로 저장
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 가중치 초기화
        self.params = {}
        #합성곱 계층
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        #Affine계층(완전연결계층)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size) #Pooling까지 마친 뒤의 크기이므로 pool_output_size
        self.params['b2'] = np.zeros(hidden_size)
        #Affine계층
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict() #key나 value에 따라 정렬 가능한 딕셔너리
        
        #합성곱계층
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        #Affine 계층
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        #추론 수행
        for layer in self.layers.values():
            #맨 앞부터 차레로 forward
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        #손실 함수
        #x : 입력 데이터
        #t : 정답 레이블
        
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

   
    def numerical_gradient(self, x, t):
        """기울기를 구한다（수치미분）.

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블

        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

     #오차역전파법으로 기울기 구하기
    def gradient(self, x, t):
        
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse() #오차역전파법 위해 layers 마지막부터 앞부분으로 backward
        for layer in layers:
            dout = layer.backward(dout)

        # 각 가중치 매개변수에 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]


In [41]:
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.trainer import Trainer


In [42]:
# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 시간이 오래 걸릴 경우 데이터를 줄인다.
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()


TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'